In [47]:
import os
import json
from ubteacher import add_ubteacher_config
from detectron2.config import get_cfg
from typing import Dict, Set, List, Tuple, Iterator
import matplotlib.pyplot as plt
import tifffile as tf

Plan: use qupath_annotations_latest instead of TissueAnnotator outputs for loading -- consistency across datasets and easier to use.
1. Load only information relevant to the task at hand from json
2. Load npy files for each image from dir, with option to create
3. Register the dataset
4. Idk

In [2]:
from ubteacher.utils.train2_utils import (get_scaling, ParseFromQuPath)

In [69]:
tissue_types = ['lymph', 'non-lymph']

In [70]:
original_file = '/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_labeled/Case 1 G7.svs'
output_file = '/mnt/RSX/Datasets_pathology/GT_2023/TissueFinderV2/SRI_OSCC/Case 1 G7.npy'
json_file = '/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_labeled/qupath_annotations_latest/Case 1 G7.json'

base_dim, target_dim = get_scaling(original_file, output_file)

dataset_dicts, cat_map = ParseFromQuPath(base_dim, target_dim, tissue_types).get_coco_format(json_file)

Image size: (33709, 39839)
Image size: (2166, 2560)


In [71]:
print(dataset_dicts)

[{'file_name': '/mnt/RSX/Datasets_pathology/GT_2023/TissueFinderV2/SRI_OSCC/Case 1 G7.npy', 'height': 2166, 'width': 2560, 'image_id': 'Case 1 G7', 'annotations': [{'category_id': 1, 'bbox_mode': 0, 'bbox': [944, 102, 2511, 1757]}, {'category_id': 1, 'bbox_mode': 0, 'bbox': [104, 698, 1407, 2105]}]}]


In [25]:
print(cat_map)

{'lymph': 0, 'non-lymph': 1}


In [26]:
print(set(cat_map.keys()))

{'lymph', 'non-lymph'}


In [ ]:
def split_dataset(self, cfg, dataset_names: List[str], args: argparse.Namespace, set_seed = False):
        """Function to collect all input datasets and split them
        into 'train' and 'val' sets.
        Args:
        dataset_names: a list of dataset names in the dataset directory
        args: parsed command-line arguments
        
        """
        data_train = dict()
        data_val = dict()
        parent_dir = cfg.PARENTDIR

        for name in dataset_names:
            image_dir = os.path.join(parent_dir, name)
            annotation_dir = os.path.join(image_dir, "tissue_annotations")

            #Try loading from dataseed
        try:
            with open(cfg.DATASEED, 'r') as f:
                data = json.load(f)
                train_set = data['train']
                val_set = data['val']
        except:
            
            print('Dataseed did not load. Creating new seed.')
                            
            train_set, val_set = self.train_val_split(
                image_dir, annotation_dir, 
            )         
        for i, j in zip((data_train, data_val), (train_set, val_set)):
            for k, v in j.items():
                if k in i:
                    i[k].extend(v)
                else:
                    i[k] = v
                        
            # Create json specifying train/val split
        if set_seed:
            data = {'train': train_set, 'val': val_set}
            with open(cfg.DATASEED, 'w') as f:
                json.dump(data, f)
                
        return data_train, data_val